In [1]:
import utils.disaster_data_utils as disaster
import utils.speech_data_utils as speech

# Ensure we use only year and country pairs that both datasets share, so we get exaclty the same length feature and target matrices.

In [2]:
# Prepare training data
df_disaster = disaster.build_clean_dataframe()
df_speech = speech.build_dataframe()

/Users/julius/repos/fundamentals_project/.venv/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:

speech_countries = set(df_speech["country"].unique())
speech_years = set(df_speech["year"].unique())
disaster_years = set(df_disaster["Year"].unique())
diaster_countries = set(df_disaster["Country"].unique())

In [4]:
# for the speech dataframes, create country year pairs
speech_country_year_pairs = []
for row in df_speech.iterrows():
    country = row[1]["country"]
    year = row[1]["year"]
    speech_country_year_pairs.append((country, year))
speech_country_year_pairs = set(speech_country_year_pairs)
    
# for the disaster dataframes, create country year pairs
disaster_country_year_pairs = []
for row in df_disaster.iterrows():
    country = row[1]["Country"]
    year = row[1]["Year"]
    disaster_country_year_pairs.append((country, year))
disaster_country_year_pairs = set(disaster_country_year_pairs)

   
   
shared_pairs = list(speech_country_year_pairs.intersection(disaster_country_year_pairs))
shared_pairs = [pair for pair in shared_pairs if pair[1] >= 1988 and pair[1] <= 2022]


# keywords = ["Co2", "climate change"]
keywords = speech.keywords

# shared_pairs = shared_pairs

In [5]:
print(len(shared_pairs))
X_train = disaster.create_feature_train_matrix(country_year_pairs=shared_pairs)
print(X_train.shape)



2056
Creating feature matrix for 2056 country-year pairs)


/Users/julius/repos/fundamentals_project/.venv/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


(2056, 4)


In [6]:
T_train = speech.create_target_value_matrix(country_year_pairs=shared_pairs, keywords=keywords)


Creating feature matrix for 2056 country-year pairs)
prepoccesing df
adding keyword counts column to df by matching keywords in stemmed stemmed text
adding year score column to df


In [7]:
T_train

array([[0.40425532],
       [0.36      ],
       [0.35064935],
       ...,
       [0.23469388],
       [0.26890756],
       [0.62068966]])

In [8]:
print(T_train.shape)
print(X_train.shape)
# # FIXME: Should be same size.
# assert(X_train.shape[0] == T_train[0])



(2056, 1)
(2056, 4)


In [9]:
# from sklearn.linear_model import LinearRegression
# # TODO: fix Nan Values
# model.fit(X_train, T_train)

# Cross Fold Validation

In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error, r2_score
model = LinearRegression(fit_intercept=True)


import numpy as np

def kfold_cross_validation(X, Y):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)
    
    # Create grid of parameters to test through cross-validation
    param_grid = {'polynomialfeatures__degree': np.arange(5),
                'linearregression__fit_intercept': [True, False]}

    model = make_pipeline(PolynomialFeatures(), LinearRegression())
    grid = GridSearchCV(model, param_grid, cv=10)
    grid.fit(X_train, Y_train);

    # Let us know check the results with the best estimator after Grid Search
    print("Best hyperparameters found:")
    print(grid.best_params_)
    y_pred = grid.best_estimator_.predict(X_test)
    print("")

    # Compute test error and variance score
    print("Model accuracy:")
    print("Mean squared error: %.2f" % mean_squared_error(Y_test, y_pred))
    print('Variance score: %.2f' % r2_score(Y_test, y_pred))
    print("")

    #Plot with test data and fitted model
    # print("Plot:")
    # x.plot.scatter(x="Freedom to make life choices", y="Perceptions of corruption",c='black')
    # xfit = np.linspace(min(X_test), max(X_test), 1000)
    # yfit = grid.best_estimator_.predict(xfit[:, np.newaxis])
    # plt.plot(xfit, yfit, c='red');

In [14]:
kfold_cross_validation(X_train, T_train)

Best hyperparameters found:
{'linearregression__fit_intercept': False, 'polynomialfeatures__degree': 0}

Model accuracy:
Mean squared error: 0.03
Variance score: -0.00



In [ ]:
# map iso code to countries
import pandas as pd
path = "../../data/CO2emissions.csv"
# create df
df = pd.read_csv(path)# Replace 'your_file.csv' with the actual path to your CSV file

# create dictionary with iso codes as keys and countries as values using Country and ISO 3166-1 alpha-3 columns
d = dict(zip(df['ISO 3166-1 alpha-3'], df["Country"]))

In [ ]:
d